# Usages of various PyTorch apis

In [1]:
import torch
import torch.nn as nn
from IPython.display import display_markdown

def print_md(s):
    display_markdown(s, raw=True)

## Embeddings
[API Docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [2]:
# This example illustrates the usage of nn.Embedding
# Embedding is a lookup table -- to lookup a vector stored against a key (typically an integer index)
# You supply a bunch of keys, you get back corresponding bunch of vectors

# This example tries to lookup vectors corresponding to words in a sentence
# Setup: here are 100 english words. Index of 'how' is 0, index of 'to' is 1, ... index of 'two' is 99
v100 = ['how', 'to', 'her', 'at', 'up', 'see', 'in', 'thing', 'even', 'because', 'or', 
             'what', 'man', 'this', 'for', 'with', 'time', 'now', 'give', 'very', 'take', 'other', 
             'there', 'would', 'first', 'about', 'people', 'think', 'find', 'so', 'say', 'as', 
             'many', 'will', 'just', 'he', 'I', 'well', 'our', 'tell', 'out', 'have', 'can', 'its', 
             'make', 'get', 'if', 'than', 'use', 'that', 'new', 'also', 'from', 'by', 'his', 'year', 
             'do', 'some', 'the', 'no', 'a', 'those', 'she', 'come', 'one', 'their', 'more', 
             'these', 'all', 'go', 'and', 'could', 'him', 'into', 'only', 'who', 'of', 'it', 'your', 
             'not', 'you', 'here', 'when', 'on', 'which', 'then', 'know', 'them', 'my', 'me', 'we', 
             'want', 'they', 'like', 'look', 'day', 'way', 'but', 'be', 'two']

# Naturally this vocabulary is insufficient even for most common sentences. Map all out of vocabulary words to <unk>
# add <unk> to vocabulary at index 100. <unk> is the replacement for out of vocabulary words
v100.append('<unk>')

# create a dictionary to lookup word's index
w2i = {word: i for i, word in enumerate(v100)}

# define few sentences of fixed size. we wish to lookup embeddings for words in these sentences
# here are 5 sentences
sents_lang = [
    'this is my book',
    'those are your books',
    'what is your name',
    'i will be back',
    'go out and about'
]

# tranform to indexed representation of sentences
sents_indices = [[w2i.get(word, w2i['<unk>']) for word in sent.split()] for sent in sents_lang]

# print representations
print_md('#### Sentences represented as array of indices')
for s1, s2 in zip(sents_lang, sents_indices):
    print_md('*{}* => {}'.format(s1, s2))

# convert sents_word_indices to a tensor
sents_tensor = torch.tensor(sents_indices)
print_md('#### Sentences represented as a tensor')
print(sents_tensor)
print('Shape = {}'.format(sents_tensor.shape))

#### Sentences represented as array of indices

*this is my book* => [13, 100, 88, 100]

*those are your books* => [61, 100, 78, 100]

*what is your name* => [11, 100, 78, 100]

*i will be back* => [100, 33, 98, 100]

*go out and about* => [69, 40, 70, 25]

#### Sentences represented as a tensor

tensor([[ 13, 100,  88, 100],
        [ 61, 100,  78, 100],
        [ 11, 100,  78, 100],
        [100,  33,  98, 100],
        [ 69,  40,  70,  25]])
Shape = torch.Size([5, 4])


In [3]:
# create an embedding to lookup 10-dimensional vectors for each word
embedding_1 = nn.Embedding(num_embeddings=len(v100), embedding_dim=10)

In [7]:
# underneath embeddings are weights, initialized with random values ~ N(0, 1)
print_md("#### Embedding weights")
print("Shape of weights = {}".format(embedding_1.weight.shape))

# what's the embedding for the word 'who'
embed = embedding_1(torch.tensor([ w2i['who']]))
print_md("#### Embedding of 'who'")
print("{}\nShape = {}".format(embed, embed.shape))

# what's the embedding of 2nd sentence
embed = embedding_1(sents_tensor[1])
print_md("#### Embedding of the sentence: *{}*".format(sents_lang[1]))
print("{}\nShape = {}".format(embed, embed.shape))

# embeddings for all sentences
embed = embedding_1(sents_tensor)
print_md("#### Embeddings of all 5 sentences")
print("{}\nShape = {}".format(embed, embed.shape))

#### Embedding weights

Shape of weights = torch.Size([101, 10])


#### Embedding of 'who'

tensor([[-1.1648, -0.4181,  0.8845,  0.8224,  0.5519, -0.2669, -0.7117,  0.2081,
          1.8748, -0.4355]], grad_fn=<EmbeddingBackward>)
Shape = torch.Size([1, 10])


#### Embedding of the sentence: *those are your books*

tensor([[-0.1838,  0.7782, -0.0468, -0.3376,  0.1745, -0.5339, -0.4489, -1.4725,
         -1.1806,  0.5460],
        [-0.7481, -0.3882, -1.2286,  2.0887, -1.7087,  1.2943,  1.5759, -1.4993,
         -0.9034, -0.8589],
        [ 2.6125, -0.5519,  1.5691, -2.0202,  0.7104, -0.7424,  0.3529,  1.6167,
          2.0495, -0.3036],
        [-0.7481, -0.3882, -1.2286,  2.0887, -1.7087,  1.2943,  1.5759, -1.4993,
         -0.9034, -0.8589]], grad_fn=<EmbeddingBackward>)
Shape = torch.Size([4, 10])


#### Embeddings of all 5 sentences

tensor([[[ 1.2721, -0.1471, -0.0531,  0.1630,  0.5624,  1.1155,  0.7727,
          -0.0911, -1.0680, -1.3912],
         [-0.7481, -0.3882, -1.2286,  2.0887, -1.7087,  1.2943,  1.5759,
          -1.4993, -0.9034, -0.8589],
         [-1.8170, -0.6491,  0.5852, -1.1165,  0.3322,  0.8839,  1.2945,
          -1.7932,  1.3616,  0.2572],
         [-0.7481, -0.3882, -1.2286,  2.0887, -1.7087,  1.2943,  1.5759,
          -1.4993, -0.9034, -0.8589]],

        [[-0.1838,  0.7782, -0.0468, -0.3376,  0.1745, -0.5339, -0.4489,
          -1.4725, -1.1806,  0.5460],
         [-0.7481, -0.3882, -1.2286,  2.0887, -1.7087,  1.2943,  1.5759,
          -1.4993, -0.9034, -0.8589],
         [ 2.6125, -0.5519,  1.5691, -2.0202,  0.7104, -0.7424,  0.3529,
           1.6167,  2.0495, -0.3036],
         [-0.7481, -0.3882, -1.2286,  2.0887, -1.7087,  1.2943,  1.5759,
          -1.4993, -0.9034, -0.8589]],

        [[ 0.3513,  0.0174, -0.4158, -1.2374, -1.3397,  0.5142, -0.2744,
           0.8178, -0.2960,  2.0155

In [5]:
# A little sophisticated embedding layer
# Suppose the embeddings we want to lookup are to be linear transformed. This can be done by implementing a custom module wrapping nn.Embedding
# This module applies a linear transformation on word embeddings of size 'in_embedding_dim', the output is embeddings of size 'out_embedding_dim'
class CustomEmbedding1(nn.Module):
    def __init__(self, num_embeddings, in_embedding_dim, out_embedding_dim):
        super(CustomEmbedding1, self).__init__()
        self.embedding = nn.Embedding(num_embeddings, in_embedding_dim)
        self.linear = nn.Linear(in_features=in_embedding_dim, out_features=out_embedding_dim)
    
    def forward(self, input):
        return self.linear(self.embedding(input))

cust_embedding_1 = CustomEmbedding1(num_embeddings=len(v100), in_embedding_dim=10, out_embedding_dim=5)

In [6]:
# what's the embedding for the word 'who'
cust_embed = cust_embedding_1(torch.tensor([ w2i['who']]))
print_md("#### Embedding of 'who'")
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

# what's the embedding of 2nd sentence
cust_embed = cust_embedding_1(sents_tensor[1])
print_md("#### Embedding of the sentence: *{}*".format(sents_lang[1]))
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

# embeddings for all sentences
cust_embed = cust_embedding_1(sents_tensor)
print_md("#### Embeddings of all 5 sentences")
print("{}\nShape = {}".format(cust_embed, cust_embed.shape))

#### Embedding of 'who'

tensor([[-0.4906, -0.3392, -0.9271,  0.0678,  1.0022]],
       grad_fn=<AddmmBackward>)
Shape = torch.Size([1, 5])


#### Embedding of the sentence: *those are your books*

tensor([[ 0.4538,  0.8740,  1.2084, -0.6495, -0.9214],
        [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483],
        [ 0.5683,  0.5978,  0.2839, -0.7738, -0.9881],
        [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483]],
       grad_fn=<AddmmBackward>)
Shape = torch.Size([4, 5])


#### Embeddings of all 5 sentences

tensor([[[ 0.5852,  1.3370,  0.3591, -0.1961, -0.7762],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483],
         [-0.5432,  0.5927, -0.5425,  0.3453,  0.5448],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483]],

        [[ 0.4538,  0.8740,  1.2084, -0.6495, -0.9214],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483],
         [ 0.5683,  0.5978,  0.2839, -0.7738, -0.9881],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483]],

        [[-0.4582,  0.4004,  0.3303,  0.7274,  0.0556],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483],
         [ 0.5683,  0.5978,  0.2839, -0.7738, -0.9881],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483]],

        [[-0.2463,  0.0895,  0.8403,  0.3683,  0.4483],
         [-0.0974, -0.7010, -0.3145,  0.3956,  0.6554],
         [-0.9131,  0.4701,  0.1388,  0.7826,  1.1020],
         [-0.2463,  0.0895,  0.8403,  0.3683,  0.4483]],

        [[ 0.1010,  1.1152, -0.2566, -0.6424, -0.5444],
         [-0.0081,  0.4487,  0.3053, -0.